# Kap. 5.3.2 Nicht lineare Prozessmodelle
## Bsp. Flash

In [1]:
from scipy.optimize import fsolve
import numpy as np

# Definition der Funktion(en)

In [2]:
# System aus MODELLGLEICHUNGEN

def F(X_var, X_para):
     
    # (Rück-)Benennung aller verwendeten Prozessgrößen
    [F, z_MeOH, z_H2O, h_F, h_MeOH_L, h_H2O_L, 
     h_MeOH_G, h_H2O_G, p_MeOH_sat, p_H2O_sat, p] = X_para
    [G, L, y_MeOH, y_H2O, x_MeOH, x_H2O, h_G, h_L, Q] = X_var   
    
    # Stoffmengenbilanzgleichungen (Typ BILANZ)
    res1 = F*z_MeOH - G*y_MeOH - L*x_MeOH           
    res2 = F*z_H2O - G*y_H2O - L*x_H2O        
    
    # Energiebilanz (Typ BILANZ)
    res3 = F*h_F - G*h_G - L*h_L + Q    
    
    # Definitionen der Gemischenthalpien (Typ KONST)
    res4 = h_G - y_MeOH*h_MeOH_G - y_H2O*h_H2O_G
    res5 = h_L - x_MeOH*h_MeOH_L - x_H2O*h_H2O_L
    
    # Definitionen des VLE (Typ KONST)
    res6 = p*y_MeOH - x_MeOH*p_MeOH_sat             
    res7 = p*y_H2O - x_H2O*p_H2O_sat 
    
    # Schließbedingungen (Typ ZWANG)
    res8 = 1 - (x_MeOH + x_H2O)    
    res9 = 1 - (y_MeOH + y_H2O) 

    # zu minimierender Residuumsvektor
    Res = [res1, res2, res3, res4, res5, res6, res7, res8, res9]
    
    return Res

# Eingabe der Werte

In [3]:
# Vorgabe (direkter und abhängiger) PARAMETER (Typ ZWANG)
# Die ausgerechneten Größen sind streng genommen Variablen. 
# Deren Gleichungen könnten ebenso in das Gleichungssystem 
# übernommen und die Größen als VARs behandelt werden.

# Charakterisierung Feed 
Feed = 25                   # Stoffmengenstrom [mol/s]
z_MeOH = 0.13               # Molanteil Methanol [-] 
z_H2O = 0.87                # " Wasser [-]

# # Bestimmung spez. Feed-Enthalpie [J/mol]
cp_MeOH = 81.08               # spez. Wärmekapazität Methanol [J/molK] 
cp_H2O = 75.29                # " Wasser [J/molK]
T_F = 35                      # Temperatur [°C]
h_F = z_MeOH*cp_MeOH*T_F + z_H2O*cp_H2O*T_F  

# # Charakterisierung des Flashs 
p = 0.15                    # Druck [bar] 
T = 50                      # Temperatur [°C]

# Verdampfungsenthalpien 
delta_h_MeOH = 35000        # [J/mol] 
delta_h_H2O = 43000         # [J/mol] 

# Berechnung spezifischer Reinstoffenthalpie der austretenden Ströme; [J/mol] 
h_MeOH_L = cp_MeOH*T                                     
h_H2O_L = cp_H2O*T                                      
h_MeOH_G = cp_MeOH*T + delta_h_MeOH                      
h_H2O_G = cp_H2O*T + delta_h_H2O                         

# Berechnung Sättigungsdampfdrücke [bar]
p_MeOH_sat = 1.013/760*10**(7.76879 - 1408.360/(T+223.600)) 
p_H2O_sat = 1.013/760*10**(8.07131 - 1730.630/(T+233.426))

X_para = [Feed, z_MeOH, z_H2O, h_F, h_MeOH_L, h_H2O_L, 
          h_MeOH_G, h_H2O_G, p_MeOH_sat, p_H2O_sat, p]

# Schaetzwerte VARIABLEN [mol/s, mol/s, - , - , - , - , J/mol, J/mol, W]
X_var_names = ['G','L','y_MeOH','y_H2O','x_MeOH','x_H2O','h_G','h_L','Q'] 
X_var_0 = [Feed*0.5, Feed*0.5, 0.5, 0.5, 0.5, 0.5, 45000, 3700, 4000000]

# Lösung durch Aufruf der Funktion(en)

In [4]:
# solver approximiert: res->0 für MODELLGLEICHUNGEN(X_var), Startwert X_var_0
X_var_final = fsolve( lambda X_var: F(X_var, X_para), X_var_0 )

# Residuumsvektor durch Einsetzen des gefundenen Lösungsvektors
Res = F(X_var_final, X_para)
# runden
Res = np.round(Res, 10)

# Untersuchung der Ergebnisse

In [8]:
# sauberes print out der Ergebnisse

print("Variablen: " + str(len(X_var_final)))
print("\n" + "X_var" + "\t" + "Wert")

for i,x in enumerate(X_var_final):
    name = X_var_names[i]
    wert = np.round(x, 3)
    print(name + "\t" + str(wert))
    
print ("\nRes = " + str (Res))

Variablen: 9

X_var	Wert
G	10.062
L	14.938
y_MeOH	0.231
y_H2O	0.769
x_MeOH	0.062
x_H2O	0.938
h_G	44985.13
h_L	3782.483
Q	442624.972

Res = [ 0. -0.  0.  0.  0.  0.  0.  0.  0.]
